Feature selection via SHAP, it like lgbm, xgb importance value. You can train some model then select columns that in top ft importance on almost model

In [ ]:
import os
import sys
from pathlib import Path

class Config:
    project_dir = Path(os.getcwd())
    logs_dir = project_dir / "logs"
    results_dir = project_dir / "results"
    data_dir = Path("/home/nikita/Data/drw-crypto-market-prediction")
    seed = 42
    
print(Config.project_dir)
print(Config.logs_dir)
print(Config.results_dir)
print(Config.data_dir)

/home/nikita/Code/kaggle
/home/nikita/Code/kaggle/logs
/home/nikita/Code/kaggle/results
/home/nikita/Data/drw-crypto-market-prediction


In [2]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import json
import os
import sys
import gc


import numpy as np
import pandas as pd
import seaborn as sns






#set pandas display options
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
# Set numpy print options
np.set_printoptions(precision=3, suppress=True, linewidth=1000)


# Load data

In [3]:
def reduce_mem_usage(dataframe, dataset):    
    print('Reducing memory usage for:', dataset)
    initial_mem_usage = dataframe.memory_usage().sum() / 1024**2
    
    for col in dataframe.columns:
        col_type = dataframe[col].dtype

        c_min = dataframe[col].min()
        c_max = dataframe[col].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                dataframe[col] = dataframe[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                dataframe[col] = dataframe[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                dataframe[col] = dataframe[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                dataframe[col] = dataframe[col].astype(np.int64)
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                dataframe[col] = dataframe[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                dataframe[col] = dataframe[col].astype(np.float32)
            else:
                dataframe[col] = dataframe[col].astype(np.float64)

    final_mem_usage = dataframe.memory_usage().sum() / 1024**2
    gc.collect()
    print('--- Memory usage before: {:.2f} MB'.format(initial_mem_usage))
    print('--- Memory usage after: {:.2f} MB'.format(final_mem_usage))
    print('--- Decreased memory usage by {:.1f}%\n'.format(100 * (initial_mem_usage - final_mem_usage) / initial_mem_usage))
    return dataframe

In [4]:
train = reduce_mem_usage(pd.read_parquet(Config.data_dir / "train.parquet"), "train")
# test = reduce_mem_usage(pd.read_parquet(Config.data_dir / "test.parquet"), "test")
# sample = pd.read_csv(Config.data_dir / "sample_submission.csv")

Reducing memory usage for: train
--- Memory usage before: 3598.94 MB
--- Memory usage after: 965.94 MB
--- Decreased memory usage by 73.2%



In [5]:
len(train)

525887

In [6]:
train.head()

/home/nikita/Code/kaggle/.venv/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X121,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X149,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X188,X189,X190,X191,X192,X193,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X303,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X381,X382,X383,X384,X385,X386,X387,X388,X389,X390,X391,X392,X393,X394,X395,X396,X397,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,X408,X409,X410,X411,X412,X413,X414,X415,X416,X417,X418,X419,X420,X421,X422,X423,X424,X425,X426,X427,X428,X429,X430,X431,X432,X433,X434,X435,X436,X437,X438,X439,X440,X441,X442,X443,X444,X445,X446,X447,X448,X449,X450,X451,X452,X453,X454,X455,X456,X457,X458,X459,X460,X461,X462,X463,X464,X465,X466,X467,X468,X469,X470,X471,X472,X473,X474,X475,X476,X477,X478,X479,X480,X481,X482,X483,X484,X485,X486,X487,X488,X489,X490,X491,X492,X493,X494,X495,X496,X497,X498,X499,X500,X501,X502,X503,X504,X505,X506,X507,X508,X509,X510,X511,X512,X513,X514,X515,X516,X517,X518,X519,X520,X521,X522,X523,X524,X525,X526,X527,X528,X529,X530,X531,X532,X533,X534,X535,X536,X537,X538,X539,X540,X541,X542,X543,X544,X545,X546,X547,X548,X549,X550,X551,X552,X553,X554,X555,X556,X557,X558,X559,X560,X561,X562,X563,X564,X565,X566,X567,X568,X569,X570,X571,X572,X573,X574,X575,X576,X577,X578,X579,X580,X581,X582,X583,X584,X585,X586,X587,X588,X589,X590,X591,X592,X593,X594,X595,X596,X597,X598,X599,X600,X601,X602,X603,X604,X605,X606,X607,X608,X609,X610,X611,X612,X613,X614,X615,X616,X617,X618,X619,X620,X621,X622,X623,X624,X625,X626,X627,X628,X629,X630,X631,X632,X633,X634,X635,X636,X637,X638,X639,X640,X641,X642,X643,X644,X645,X646,X647,X648,X649,X650,X651,X652,X653,X654,X655,X656,X657,X658,X659,X660,X661,X662,X663,X664,X665,X666,X667,X668,X669,X670,X671,X672,X673,X674,X675,X676,X677,X678,X679,X680,X681,X682,X683,X684,X685,X686,X687,X688,X689,X690,X691,X692,X693,X694,X695,X696,X697,X698,X699,X700,X701,X702,X703,X704,X705,X706,X707,X708,X709,X710,X711,X712,X713,X714,X715,X716,X717,X718,X719,X720,X721,X722,X723,X724,X725,X726,X727,X728,X729,X730,X731,X732,X733,X734,X735,X736,X737,X738,X739,X740,X741,X742,X743,X744,X745,X746,X747,X748,X749,X750,X751,X752,X753,X754,X755,X756,X757,X758,X759,X760,X761,X762,X763,X764,X765,X766,X767,X768,X769,X770,X771,X772,X773,X774,X775,X776,X777,X778,X779,X780,X781,X782,X783,X784,X785,X786,X787,X788,X789,X790,X791,X792,X793,X794,X795,X796,X797,X798,X799,X800,X801,X802,X803,X804,X805,X806,X807,X808,X809,X810,X811,X812,X813,X8

In [7]:
train.mean(axis=0)

/home/nikita/Code/kaggle/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1496: RuntimeWarning: overflow encountered in cast
  return count.astype(dtype, copy=False)
/home/nikita/Code/kaggle/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:51: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


bid_qty     NaN
ask_qty     NaN
buy_qty     NaN
sell_qty    NaN
volume      NaN
X1         -0.0
X2         -0.0
X3         -0.0
X4         -0.0
X5         -0.0
X6         -0.0
X7         -0.0
X8         -0.0
X9          0.0
X10        -0.0
X11        -0.0
X12        -0.0
X13        -0.0
X14        -0.0
X15        -0.0
X16        -0.0
X17         NaN
X18        -0.0
X19        -0.0
X20        -0.0
X21        -0.0
X22        -0.0
X23        -0.0
X24        -0.0
X25         NaN
X26        -0.0
X27        -0.0
X28        -0.0
X29        -0.0
X30        -0.0
X31        -0.0
X32        -0.0
X33         NaN
X34         NaN
X35         NaN
X36         NaN
X37         NaN
X38         NaN
X39         NaN
X40         NaN
X41         NaN
X42         NaN
X43         NaN
X44         NaN
X45         NaN
X46         NaN
X47         NaN
X48         NaN
X49         NaN
X50         NaN
X51         NaN
X52         NaN
X53         NaN
X54         NaN
X55         NaN
X56         NaN
X57         0.0
X58     

# Remove constant columns

In [ ]:
# # Constant features
# constant_cols = [col for col in train.columns if train[col].nunique() == 1]

In [ ]:
# constant_cols

In [ ]:
# train.drop(columns=constant_cols, inplace=True)

# Pairwise correlation

In [ ]:

# # Assuming your DataFrame is named df and the target column is 'target'
# # Drop the target column to focus only on features
# feature_df = train.drop(columns=['label'], errors='ignore').iloc[::5]

# # Compute pairwise Pearson correlation matrix
# correlation_matrix = feature_df.corr(method='pearson')

# # Display shape and sample
# print("Correlation matrix shape:", correlation_matrix.shape)
# correlation_matrix.head()

In [ ]:
# # Mask upper triangle to avoid duplicate pairs and self-correlation
# mask = np.triu(np.ones(correlation_matrix.shape), k=1)
# corr_matrix_masked = correlation_matrix.where(mask == 1)

# corr_pairs = corr_matrix_masked[
#     (0.995 < corr_matrix_masked.abs())# & (corr_matrix_masked.abs() < 1)
# ].stack().reset_index()

# # Rename columns for clarity
# corr_pairs.columns = ['feature_1', 'feature_2', 'correlation']

# # Show result
# print(f"Found {len(corr_pairs)} feature pairs.")
# corr_pairs

In [ ]:
# pairs = {}
# used = set()
# for v1, v2 in zip(corr_pairs['feature_1'], corr_pairs['feature_2']):
#     if v2 not in used:
#         if v1 not in pairs:
#             pairs[v1] = [v1, v2]
#             used.add(v1)
#             used.add(v2)
#         else:
#             pairs[v1] += [v2]
#             used.add(v2)
# pairs

In [ ]:
# copies_to_drop = []
# str_to_drop = ""
# for k, v in pairs.items():
#     copies_to_drop += v[1:]
#     str_to_drop += ("'"+"', '".join(v) + "', ")
# print(f"Number of features to drop: {len(copies_to_drop)}")
# str_to_drop

In [ ]:
# train.drop(columns=copies_to_drop, inplace=True)

In [ ]:
# len(train.columns)

# Final features to drop

In [8]:
high_corr_drop = ['X39', 'X41', 'X40', 'X42', 'X45', 'X47', 'X46', 'X48', 'X47', 
                  'X49', 'X48', 'X50', 'X51', 'X53', 'X55', 'X52', 'X54', 'X56', 
                  'X62', 'X104', 'X146', 'X68', 'X110', 'X152', 'X74', 'X116', 
                  'X158', 'X80', 'X122', 'X164', 'X86', 'X128', 'X170', 'X92', 
                  'X134', 'X176', 'X98', 'X140', 'X182', 'X234', 'X241', 'X235', 
                  'X242', 'X236', 'X243', 'X237', 'X244', 'X238', 'X245', 'X239', 
                  'X246', 'X280', 'X282', 'X281', 'X283', 'X286', 'X288', 'X287', 
                  'X289', 'X288', 'X290', 'X292', 'X294', 'X293', 'X295', 'X294', 
                  'X296', 'X295', 'X297', 'X298', 'X300', 'X302', 'X299', 'X301', 
                  'X303', 'X309', 'X351', 'X393', 'X315', 'X357', 'X399', 'X321', 
                  'X363', 'X405', 'X327', 'X369', 'X411', 'X333', 'X375', 'X417', 
                  'X339', 'X381', 'X423', 'X345', 'X387', 'X429', 'X481', 'X488', 
                  'X482', 'X489', 'X483', 'X490', 'X484', 'X491', 'X485', 'X492', 
                  'X486', 'X493', 'X487', 'X494', 'X613', 'X619', 'X616', 'X622', 
                  'X625', 'X631', 'X628', 'X634', 'X637', 'X643', 'X640', 'X646', 
                  'X649', 'X655', 'X652', 'X658', 'X661', 'X667', 'X664', 'X670', 
                  'X673', 'X679', 'X676', 'X682', 'X685', 'X691', 'X688', 'X694']

constant_cols = ['X697', 'X698', 'X699', 'X700', 'X701', 'X702', 'X703', 'X704', 'X705', 
                 'X706', 'X707', 'X708', 'X709', 'X710', 'X711', 'X712', 'X713', 'X714', 
                 'X715', 'X716', 'X717', 'X864', 'X867', 'X869', 'X870', 'X871', 'X872']

train.drop(columns=high_corr_drop + constant_cols, inplace=True)

In [9]:
train.head()

/home/nikita/Code/kaggle/.venv/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X43,X44,X57,X58,X59,X60,X61,X63,X64,X65,X66,X67,X69,X70,X71,X72,X73,X75,X76,X77,X78,X79,X81,X82,X83,X84,X85,X87,X88,X89,X90,X91,X93,X94,X95,X96,X97,X99,X100,X101,X102,X103,X105,X106,X107,X108,X109,X111,X112,X113,X114,X115,X117,X118,X119,X120,X121,X123,X124,X125,X126,X127,X129,X130,X131,X132,X133,X135,X136,X137,X138,X139,X141,X142,X143,X144,X145,X147,X148,X149,X150,X151,X153,X154,X155,X156,X157,X159,X160,X161,X162,X163,X165,X166,X167,X168,X169,X171,X172,X173,X174,X175,X177,X178,X179,X180,X181,X183,X184,X185,X186,X187,X188,X189,X190,X191,X192,X193,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X240,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X284,X285,X291,X304,X305,X306,X307,X308,X310,X311,X312,X313,X314,X316,X317,X318,X319,X320,X322,X323,X324,X325,X326,X328,X329,X330,X331,X332,X334,X335,X336,X337,X338,X340,X341,X342,X343,X344,X346,X347,X348,X349,X350,X352,X353,X354,X355,X356,X358,X359,X360,X361,X362,X364,X365,X366,X367,X368,X370,X371,X372,X373,X374,X376,X377,X378,X379,X380,X382,X383,X384,X385,X386,X388,X389,X390,X391,X392,X394,X395,X396,X397,X398,X400,X401,X402,X403,X404,X406,X407,X408,X409,X410,X412,X413,X414,X415,X416,X418,X419,X420,X421,X422,X424,X425,X426,X427,X428,X430,X431,X432,X433,X434,X435,X436,X437,X438,X439,X440,X441,X442,X443,X444,X445,X446,X447,X448,X449,X450,X451,X452,X453,X454,X455,X456,X457,X458,X459,X460,X461,X462,X463,X464,X465,X466,X467,X468,X469,X470,X471,X472,X473,X474,X475,X476,X477,X478,X479,X480,X495,X496,X497,X498,X499,X500,X501,X502,X503,X504,X505,X506,X507,X508,X509,X510,X511,X512,X513,X514,X515,X516,X517,X518,X519,X520,X521,X522,X523,X524,X525,X526,X527,X528,X529,X530,X531,X532,X533,X534,X535,X536,X537,X538,X539,X540,X541,X542,X543,X544,X545,X546,X547,X548,X549,X550,X551,X552,X553,X554,X555,X556,X557,X558,X559,X560,X561,X562,X563,X564,X565,X566,X567,X568,X569,X570,X571,X572,X573,X574,X575,X576,X577,X578,X579,X580,X581,X582,X583,X584,X585,X586,X587,X588,X589,X590,X591,X592,X593,X594,X595,X596,X597,X598,X599,X600,X601,X602,X603,X604,X605,X606,X607,X608,X609,X610,X611,X612,X614,X615,X617,X618,X620,X621,X623,X624,X626,X627,X629,X630,X632,X633,X635,X636,X638,X639,X641,X642,X644,X645,X647,X648,X650,X651,X653,X654,X656,X657,X659,X660,X662,X663,X665,X666,X668,X669,X671,X672,X674,X675,X677,X678,X680,X681,X683,X684,X686,X687,X689,X690,X692,X693,X695,X696,X718,X719,X720,X721,X722,X723,X724,X725,X726,X727,X728,X729,X730,X731,X732,X733,X734,X735,X736,X737,X738,X739,X740,X741,X742,X743,X744,X745,X746,X747,X748,X749,X750,X751,X752,X753,X754,X755,X756,X757,X758,X759,X760,X761,X762,X763,X764,X765,X766,X767,X768,X769,X770,X771,X772,X773,X774,X775,X776,X777,X778,X779,X780,X781,X782,X783,X784,X785,X786,X787,X788,X789,X790,X791,X792,X793,X794,X795,X796,X797,X798,X799,X800,X801,X802,X803,X804,X805,X806,X807,X808,X809,X810,X811,X812,X813,X814,X815,X816,X817,X818,X819,X820,X821,X822,X823,X824,X825,X826,X827,X828,X829,X830,X831,X832,X833,X834,X835,X836,X837,X838,X839,X840,X841,X842,X843,X844,X845,X846,X847,X848,X849,X850,X851,X852,X853,X854,X855,X856,X857,X858,X859,X860,X861,X862,X863,X865,X866,X868,X873,X874,X875,X876,X877,X878,X879,X880,X881,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

# feature importance

# cv code

In [ ]:
# What do i want to do here? 
from sklearn.base import BaseEstimator
from sklearn.model_selection import KFold


def xgb_params(trial):
    return {
        # fixed
        'objective': trial.suggest_categorical('objective', ['reg:squarederror']),
        'tree_method': trial.suggest_categorical('objective', ['hist']),
        'device': trial.suggest_categorical('device', ['cuda']),
        'random_state': trial.suggest_categorical('random_state', [Config.seed]),
        # hyperparams
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'subsample': trial.suggest_float('subsample', 0.01, 0.5, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.7),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 0.7),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.3, 0.7),
        'lambda': trial.suggest_float('lambda', 10, 200, log=True),
        'alpha': trial.suggest_float('alpha', 10, 100, log=True),
        'gamma': trial.suggest_float('gamma', 0.0, 3.0),
    }# other: n_jobs, random_state, verbose, max_leaves, min_child_weight



def lgbm_params(trial):
    return {
        # fixed
        'objective': trial.suggest_categorical('objective', ['regression']),
        'device': trial.suggest_categorical('device', ['cuda']),
        'random_state': trial.suggest_categorical('random_state', [Config.seed]),
        # hyperparams
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128),
        'subsample': trial.suggest_float('subsample', 0.01, 0.5, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.7),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.3, 0.7),
        'lambda_l1': trial.suggest_float('lambda_l1', 10, 100, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 10, 200, log=True),
    }# other? goss


def catb_params(trial):
    return {
        # fixed
        'objective': trial.suggest_categorical('objective', ['RMSE']),
        'task_type': trial.suggest_categorical('task_type', ['GPU']),
        'random_state': trial.suggest_categorical('random_state', [Config.seed]),
        # hyperparams
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'depth': trial.suggest_int('depth', 3, 7),
        'subsample': trial.suggest_float('subsample', 0.01, 0.5, log=True),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 0.7),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.5, 2.0),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 10, 200, log=True),
    }# other? max_bin ?




# ive decided to use naive nested 5 fold to do the tuning. 
# maybe setting one of them to None can yield back simple kfold. 
# not sure what to do yet for ensembling, but this is an issue for way later
def do_optuna_optimization(
    X: np.ndarray,
    y: np.ndarray, 
    model: BaseEstimator,
    params_fn: Callable = xgb_params,
    n_trials: int = 100,
    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42),
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=42),
):
    pass
    